### The Case

It´s your first day in a Data Science advisory firm and your boss asks you to produce the __official Summer Olympic Games Medal Tables for all Editions from 1896 to 2012__. <br><br>
All you can use is a dataset with raw data containing over 31,000 medals (__summer.csv__) and the official Medal Tables for the Editions 1996 and 1976 from Wikipedia. (__wik_1996.csv__, __wik_1976.csv__). Use the two official Medal Tables as a __reference__ to check whether your code produces the correct output! <br><br>
Your goal is to __minimize the divergence__ between your aggregated Medal Tables and the official Medal Tables. Let´s assume that the official number of Gold Medals for the United States in the Edition 1996 is 44 and your code produces 46. This is an absolute divergence of 2. <br> <br>
__Calculate the total absolute divergence for the Editions 1996 and 1976 (the "Score")!__ The __optimal Score is 0__! 

In [1]:
import pandas as pd
import numpy as np

In [2]:
summer = pd.read_csv("summer.csv")
wik_1996 = pd.read_csv("wik_1996.csv")
wik_1976 = pd.read_csv("wik_1976.csv")

### INITIAL EXPLORATION

In [3]:
summer.head()

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
0,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100M Freestyle,Gold
1,1896,Athens,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100M Freestyle,Silver
2,1896,Athens,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100M Freestyle For Sailors,Bronze
3,1896,Athens,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100M Freestyle For Sailors,Gold
4,1896,Athens,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100M Freestyle For Sailors,Silver


In [4]:
#We have a few Nans here
summer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31165 entries, 0 to 31164
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Year        31165 non-null  int64 
 1   City        31165 non-null  object
 2   Sport       31165 non-null  object
 3   Discipline  31165 non-null  object
 4   Athlete     31165 non-null  object
 5   Country     31161 non-null  object
 6   Gender      31165 non-null  object
 7   Event       31165 non-null  object
 8   Medal       31165 non-null  object
dtypes: int64(1), object(8)
memory usage: 2.1+ MB


In [5]:
wik_1976.head()

,Rank,NOC,Gold,Silver,Bronze,Total
0,1,Soviet Union (URS),49,41,35,125
1,2,East Germany (GDR),40,25,25,90
2,3,United States (USA),34,35,25,94
3,4,West Germany (FRG),10,12,17,39
4,5,Japan (JPN),9,6,10,25


In [6]:
#no hay nulls
wik_1976.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Rank    42 non-null     object
 1   NOC     42 non-null     object
 2   Gold    42 non-null     int64 
 3   Silver  42 non-null     int64 
 4   Bronze  42 non-null     int64 
 5   Total   42 non-null     int64 
dtypes: int64(4), object(2)
memory usage: 2.1+ KB


In [7]:
wik_1996.head()

,Rank,Nation,Gold,Silver,Bronze,Total
0,1,United States (USA)*,44,32,25,101
1,2,Russia (RUS),26,21,16,63
2,3,Germany (GER),20,18,27,65
3,4,China (CHN),16,22,12,50
4,5,France (FRA),15,7,15,37


In [8]:
wik_1996.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Rank    80 non-null     object
 1   Nation  80 non-null     object
 2   Gold    80 non-null     int64 
 3   Silver  80 non-null     int64 
 4   Bronze  80 non-null     int64 
 5   Total   80 non-null     int64 
dtypes: int64(4), object(2)
memory usage: 3.9+ KB


### INITIAL COLUMNS MODIFICATION

#### Extracting the country code from the Nation and NOC columns

In [9]:
wik_1976['NOC'] = wik_1976['NOC'].str.split('(', expand=True).iloc[:,1].str.replace(')','', regex = True).str.replace('*','',regex = True)

In [10]:
wik_1996['Nation'] = wik_1996['Nation'].str.split('(', expand=True).iloc[:,1].str.replace(')','', regex = True).str.replace('*','',regex = True)

In [11]:
wik_1976.rename(columns={'NOC':'Country'}, inplace=True)

In [12]:
wik_1996.rename(columns={'Nation':'Country'}, inplace=True)

In [13]:
summer['Event_Gender'] = summer['Gender']

In [14]:
wik_1976 = wik_1976.drop(columns=['Rank','Total']).set_index('Country').sort_values(by = ['Gold','Silver','Bronze'], ascending = False )
wik_1976.head()

,Gold,Silver,Bronze
Country,,,
41 NOCs,198,199,216
URS,49,41,35
GDR,40,25,25
USA,34,35,25
FRG,10,12,17


In [15]:
wik_1976.drop(labels='41 NOCs',inplace=True)

In [16]:
wik_1996 = wik_1996.drop(columns=['Rank','Total']).set_index('Country').sort_values(by = ['Gold','Silver','Bronze'], ascending = False )
wik_1996.head()

,Gold,Silver,Bronze
Country,,,
79 nations,271,273,298
USA,44,32,25
RUS,26,21,16
GER,20,18,27
CHN,16,22,12


In [17]:
wik_1996.drop(labels='79 nations',inplace=True)

### FIND MIXED EVENTS

In a first step, we need to determine for each row / medal, whether the medal was awarded in a Men, Women or Mixed Event. <br>
The default assumption: The values in the new column Event_Gender are the same as in the column Gender (this is the Gender of the respective Athlete). Then, we need to identify Mixed Gender Events (inputs from the experts!).

In [18]:
mask1 = summer['Event'].str.contains('Mixed')

In [19]:
mask2 = summer['Event'].str.contains('Pairs')

In [20]:
mask3 = summer['Sport'].str.contains('Equestrian')

In [21]:
mask4 = (summer['Sport'].str.contains('Sail')) & (summer['Year'] < 1988)

In [22]:
mask5 = [21773, 21782, 21776,21785, 21770, 21779,23703,23712,23706, 23715,23709,23700,25720,25729,25723,25732,25726,
              25717,27727,27736, 27730,27739,27724,27733, 29784, 29785,29786,29787,29788,29789]

In [23]:
summer.loc[mask1|mask2|mask3|mask4|summer.index.isin(mask5), 'Event_Gender'] = 'X'

In [24]:
summer['Event_Gender'].value_counts()

Men      21214
Women     8177
X         1774
Name: Event_Gender, dtype: int64

### IDENTIFY UNIQUE EVENTS

Unique events are those that match in Year, Sport, Discipline, Event and Event Gender.

In [25]:
#lets make a column with the number of Medals by Event.
summer['Event_Medals'] = pd.Series(summer.groupby(['Year','Sport','Discipline','Event','Event_Gender'])['Medal'].transform('count'))

In [26]:
summer['Team'] = pd.Series(np.where(summer['Event_Medals'] >5, 'Yes','No'))

In [27]:
summer.reset_index(inplace=True)

In [28]:
teams = summer.loc[summer['Team'] =='Yes'].copy()

In [29]:
singles = summer.loc[summer['Team'] =='No'].copy()

### DROP DUPLICATES 

We will drop duplicated in teams df, which are ones were the subset is equal in the following columns ['Year','Sport','Discipline','Event','Event_Gender']

In [30]:
teams.drop_duplicates(subset=['Year','Sport','Discipline','Event','Event_Gender'],ignore_index=True)

,index,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal,Event_Gender,Event_Medals,Team
0,84,1896,Athens,Gymnastics,Artistic G.,"BÖCKER, Konrad",GER,Men,"Team, Horizontal Bar",Gold,Men,11,Yes
1,95,1896,Athens,Gymnastics,Artistic G.,"CHRYSAPHIS, Ioannis",GRE,Men,"Team, Parallel Bars",Bronze,Men,19,Yes
2,132,1896,Athens,Tennis,Tennis,"FLACK, Edwin",ZZX,Men,Doubles,Bronze,Men,6,Yes
3,163,1900,Paris,Aquatics,Swimming,HOUBEN,FRA,Men,200M Team Swimming,Bronze,Men,15,Yes
4,184,1900,Paris,Aquatics,Water polo,"BURGESS, Thomas",FRA,Men,Water Polo,Bronze,Men,28,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,30946,2012,London,Tennis,Tennis,"AZARENKA, Victoria",BLR,Women,Mixed Doubles,Gold,X,6,Yes
1232,30964,2012,London,Volleyball,Beach Volleyball,"BRINK, Julius",GER,Men,Beach Volleyball,Gold,Men,6,Yes
1233,30970,2012,London,Volleyball,Beach Volleyball,"MAY, Misty",USA,Women,Beach Volleyball,Gold,Women,6,Yes
1234,30976,2012,London,Volleyball,Volleyball,"APALIKOV, Nikolay",RUS,Men,Volleyball,Gold,Men,36,Yes


### CONCAT BOTH DF AGAIN

In [31]:
summer_new = pd.concat([singles,teams],)

In [32]:
summer.reset_index(inplace=True)
summer_new.head()

,index,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal,Event_Gender,Event_Medals,Team
0,0,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100M Freestyle,Gold,Men,2,No
1,1,1896,Athens,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100M Freestyle,Silver,Men,2,No
2,2,1896,Athens,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100M Freestyle For Sailors,Bronze,Men,3,No
3,3,1896,Athens,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100M Freestyle For Sailors,Gold,Men,3,No
4,4,1896,Athens,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100M Freestyle For Sailors,Silver,Men,3,No


#### Lets create a medal Tables where we have a multiindex with the countries and the year

In [33]:
medal_tables = summer_new.groupby(['Year','Country','Medal'])['Medal'].count().unstack(fill_value = 0)[['Gold','Silver','Bronze']]
medal_tables.head()

Medal         Gold  Silver  Bronze
Year Country                      
1896 AUS         2       0       0
     AUT         2       1       2
     DEN         1       2       3
     FRA         5       4       2
     GBR         2       3       2

### LETS GET THE 1976 AND 1996 DFS From the Medal_Tables df

In [34]:
agg_1976 = medal_tables.loc[1976].sort_values(by = ['Gold','Silver','Bronze'], ascending = False)
agg_1976.head()

Medal,Gold,Silver,Bronze
Country,,,
URS,113,93,79
GDR,99,51,42
USA,63,56,36
JPN,25,6,10
FRG,21,24,30


In [35]:
agg_1996 = medal_tables.loc[1996].sort_values(by = ['Gold','Silver','Bronze'], ascending = False )
agg_1996.head()

Medal,Gold,Silver,Bronze
Country,,,
USA,160,48,52
GER,42,35,47
CUB,39,8,10
NED,38,9,26
RUS,36,45,34


In [42]:
div76 = agg_1976.sub(wik_1976).abs().dropna().head()

In [43]:
score_76 = div76.sum().sum()
score_76

23

In [44]:
wik_1996

,Gold,Silver,Bronze
Country,,,
USA,44,32,25
RUS,26,21,16
GER,20,18,27
CHN,16,22,12
FRA,15,7,15
...,...,...,...
MGL,0,0,1
MOZ,0,0,1
PUR,0,0,1


In [41]:
div96 = agg_1996.sub(wik_1996).abs().dropna().head()
div96

Medal,Gold,Silver,Bronze
Country,,,
ALG,0,0,0
ARG,0,17,0
ARM,0,0,0
AUS,23,7,61
AUT,0,0,0


In [40]:
score_96 = div96.sum().sum()
score_96

108